In [1]:
import utils

	
 !"#%&'()*+,-./0123456789:<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]_abcdefghijklmnopqrstuvwxyz{}ĆćčđŠšžНУабвгдезиклмнопрстухчшђјњћ…⁠✋❤﻿🏻🏼🏽👋👌👍👽💪😀😂😃😍😟🙂🙌🙏🤑🤘🤙🤝🤢🤣🤮🥰🥷🥹🧠🫂🫶
159
Step 0: train loss 5.4162, val loss 5.4183
Step 1000: train loss 1.6744, val loss 1.8614
Step 2000: train loss 1.5953, val loss 1.7532
Step 3000: train loss 1.5668, val loss 1.7057
Step 4000: train loss 1.5169, val loss 1.6890
Step 5000: train loss 1.5025, val loss 1.6680
Step 6000: train loss 1.4982, val loss 1.6675
Step 7000: train loss 1.4742, val loss 1.6535
Step 8000: train loss 1.4452, val loss 1.6646
Step 9000: train loss 1.4857, val loss 1.6254


In [2]:
utils.intuition_test('ee brate')

ee brate na sonj
[12/27/2025 4:24 PM5 Filip Trivan: laceg
[1-22/2025 5:51 PM] Aleksa Trivan: njiga tecmo imi
